In [ ]:
!pip install transformers datasets accelerate librosa ffmpeg-python soundfile gradio==3.50.2 reportlab

In [ ]:
import librosa, numpy as np, ffmpeg, os, uuid, cv2, base64, io
import IPython.display as ipd
import matplotlib.pyplot as plt
import gradio as gr
from transformers import pipeline
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

In [ ]:
ASR_MODEL = "openai/whisper-small"
# After your training:
# ASR_MODEL = "prob12/mentorai-whisper-indian"

asr = pipeline("automatic-speech-recognition", model=ASR_MODEL, chunk_length_s=20)

Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [ ]:
def extract_engagement(video_path):
    cap = cv2.VideoCapture(video_path)
    eye_contact_frames = 0
    total_frames = 0
    face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        total_frames += 1

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray,1.2,5)
        if len(faces) > 0:
            eye_contact_frames += 1

    cap.release()

    if total_frames == 0:
        return 5.0

    engagement = round((eye_contact_frames/total_frames)*10, 2)
    return engagement


In [ ]:
def evaluate_teaching(text, audio_len, video_path=None):

    words = len(text.split())
    wpm = (words / audio_len) * 60

    filler = sum(text.lower().count(w) for w in ["um", "uh", "basically", "actually"])

    # 1. Clarity
    clarity = max(1, 10 - filler * 1.2)

    # 2. Pace
    pace = max(1, min(10, wpm / 13))

    # 3. Richness
    richness = min(10, words / 50)

    # 4. Confidence (pauses)
    confidence = max(1, min(10, 10 - abs(wpm - 130) / 13))

    # 5. Engagement (from video if available)
    engagement = extract_engagement(video_path) if video_path else 7.0

    final = round(
        clarity*0.25 + pace*0.2 + richness*0.2 + confidence*0.2 + engagement*0.15, 2
    )

    return {
        "clarity": round(clarity,2),
        "pace": round(pace,2),
        "richness": round(richness,2),
        "confidence": round(confidence,2),
        "engagement": round(engagement,2),
        "final_score": final
    }


In [ ]:
def extract_audio(video_path, out_path="audio_temp.wav"):
    (
        ffmpeg.input(video_path)
              .output(out_path, ac=1, ar="16000")
              .overwrite_output()
              .run(quiet=True)
    )
    return out_path


In [ ]:
def segment_audio(audio_path, chunk_sec=10):
    audio, sr = librosa.load(audio_path, sr=16000)
    total = len(audio)
    segs = []
    samples = chunk_sec * sr

    for start in range(0, total, samples):
        end = min(start + samples, total)
        seg = audio[start:end]
        seg_path = f"seg_{uuid.uuid4()}.wav"
        save_audio_segment(seg, sr, seg_path)
        segs.append((seg_path, start/sr, end/sr))
    return segs


def micro_feedback(audio_path):
    segments = segment_audio(audio_path, 8)
    feedback = []
    for seg_path, start, end in segments:
        tx = whisper_transcribe(seg_path, asr) # Changed to use whisper_transcribe
        feedback.append({
            "start": round(start,2),
            "end": round(end,2),
            "text": tx
        })
    return feedback

In [ ]:
def analyze_file(file_path):

    ext = file_path.split('.')[-1].lower()
    if ext in ["mp4","mov","mkv","avi"]:
        audio_path = extract_audio(file_path)
        video_path = file_path
    else:
        audio_path = file_path
        video_path = None

    audio, sr = librosa.load(audio_path, sr=16000)
    duration = len(audio) / sr

    text = whisper_transcribe(audio_path, asr) # Changed to use whisper_transcribe

    scores = evaluate_teaching(text, duration, video_path)

    micro = micro_feedback(audio_path)

    return text, scores, duration, micro

In [ ]:
def generate_pdf_report(text, scores, micro, filename="report.pdf"):
    c = canvas.Canvas(filename, pagesize=letter)
    c.setFont("Helvetica", 12)
    c.drawString(30, 750, "MentorAI — Teaching Evaluation Report")
    c.drawString(30, 730, f"Final Score: {scores['final_score']} / 10")
    c.drawString(30, 710, f"Clarity: {scores['clarity']}")
    c.drawString(30, 690, f"Pace: {scores['pace']}")
    c.drawString(30, 670, f"Confidence: {scores['confidence']}")
    c.drawString(30, 650, f"Engagement: {scores['engagement']}")

    y = 620
    c.drawString(30, y, "Micro-Feedback:")
    y -= 20

    for seg in micro[:10]:
        c.drawString(30, y, f"{seg['start']}s - {seg['end']}s: {seg['text'][:70]}")
        y -= 20
        if y < 50:
            c.showPage()
            y = 750

    c.save()
    return filename


In [ ]:
def live_record(seconds=10):
    from google.colab import output
    audio = output.record_audio(duration=seconds)
    b = audio.split(",")[1]
    wav = base64.b64decode(b)
    fn = f"live_{uuid.uuid4()}.wav"
    with open(fn, "wb") as f:
        f.write(wav)
    return fn


def live_analyze(seconds=10):
    path = live_record(seconds)
    text, scores, duration, micro = analyze_file(path)
    return text, scores, f"Recorded {seconds}s"


In [ ]:
def ui_upload(file):
    file_path = safe_file_path(file);
    if not file_path:
        return "❌ No file uploaded!", None;

    # Reverted to direct call, as this was working before
    text, scores, duration, micro = analyze_file(file_path);

    pdf = generate_pdf_report(text, scores, micro);

    output = f"""
# 🧑‍🏫 MentorAI Teaching Evaluation Report
### **Final Score: {scores['final_score']} / 10**

**Clarity:** {scores['clarity']}
**Pace:** {scores['pace']}
**Confidence:** {scores['confidence']}
**Engagement:** {scores['engagement']}

---

### **Transcript**
{text}

---

### **Micro Feedback (First 5 segments)**
"""
    for m in micro[:5]:
        output += f"- **{m['start']}s → {m['end']}s:** {m['text']} \n";

    return output, pdf;


with gr.Blocks(title="MentorAI — Full AI Teaching Evaluation Suite") as demo:
    gr.Markdown("# 🎓 MentorAI — Complete Teaching Evaluation System");
    gr.Markdown("Upload video/audio OR test with your microphone.");

    with gr.Tab("Upload Mode"):
        file = gr.File(label="Upload Video or Audio");
        out = gr.Markdown();
        pdf_out = gr.File();
        btn = gr.Button("Analyze");
        btn.click(ui_upload, inputs=file, outputs=[out, pdf_out]);

    with gr.Tab("Live Mode"):
        audio_input_live = gr.Audio(source="microphone", type="filepath");
        live_out = gr.Markdown();
        lbtn = gr.Button("Start Live Evaluation");
        # Keep debug_wrapper for live mode, but analyze_live_gradio_input will have its own try-except
        lbtn.click(lambda x: debug_wrapper(analyze_live_gradio_input, x), inputs=audio_input_live, outputs=live_out);

demo.launch(**GRADIO_LAUNCH_KWARGS);

IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://26234d0cdc9f367834.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# ======================================================
#  FULL LIVE RECORDING + ANALYSIS + GRADIO UI (WORKING)
# ======================================================

!pip install gradio==3.50.2 soundfile librosa transformers accelerate ffmpeg-python

import base64, uuid, librosa, numpy as np, ffmpeg, os
from google.colab import output
import gradio as gr
from transformers import pipeline

# ============================
# LOAD WHISPER MODEL
# ============================
ASR_MODEL = "openai/whisper-small"
asr = pipeline("automatic-speech-recognition", model=ASR_MODEL, chunk_length_s=20)

# ============================
# LIVE RECORD FUNCTION
# ============================
def live_record(seconds=10):
    print("🎤 Recording... allow microphone permission when asked.")
    try:
        audio_bytes = output.record_audio(duration=seconds)
        if audio_bytes is None:
            return None, "Microphone permission denied."

        # Decode base64 audio
        audio_str = audio_bytes.split(",")[1]
        audio_decoded = base64.b64decode(audio_str)

        # Save file
        filename = f"live_{uuid.uuid4()}.wav"
        with open(filename, "wb") as f:
            f.write(audio_decoded)

        return filename, "OK"
    except Exception as e:
        return None, f"Recording error: {str(e)}"


# ============================
# SIMPLE TEACHING SCORING LOGIC
# ============================
def score_teaching(text, duration):
    words = len(text.split())
    wpm = (words / duration) * 60 if duration > 0 else 1

    clarity = max(1, min(10, 10 - (text.lower().count("um") + text.lower().count("uh"))))
    pace = max(1, min(10, wpm / 13))
    richness = min(10, words / 50)
    confidence = max(1, min(10, 10 - abs(wpm - 130) / 13))

    final = round((clarity*0.4 + pace*0.2 + richness*0.2 + confidence*0.2), 2)

    return {
        "clarity": round(clarity, 2),
        "pace": round(pace, 2),
        "richness": round(richness, 2),
        "confidence": round(confidence, 2),
        "final": final
    }


# ============================
# LIVE ANALYZE FUNCTION
# ============================
def live_analyze(seconds):

    # Step 1: Record
    file_path, status = live_record(seconds)
    if file_path is None:
        return f"❌ Error: {status}"

    # Step 2: Transcribe
    result = asr(file_path)
    text = result["text"]

    # Step 3: Get duration
    audio, sr = librosa.load(file_path, sr=16000)
    duration = len(audio) / sr

    # Step 4: Score
    scores = score_teaching(text, duration)

    # Step 5: Return formatted output
    return f"""
# 🎤 Live Teaching Evaluation

### ⏱ Duration: {round(duration,2)} sec
### 📝 Transcript:
{text}

---

### ⭐ Scores
- **Clarity:** {scores['clarity']}
- **Pace:** {scores['pace']}
- **Richness:** {scores['richness']}
- **Confidence:** {scores['confidence']}

### 🎯 Final Score: **{scores['final']} / 10**
"""


# ============================
# GRADIO UI FOR LIVE MODE
# ============================
with gr.Blocks(title="Mentor AI - Live Teaching Evaluation") as demo:

    gr.Markdown("# 🎤 Mentor AI — Live Teaching Evaluation")
    gr.Markdown("Record your voice for 5–30 seconds and get instant AI feedback.")

    duration = gr.Slider(5, 30, value=10, label="Record Duration (seconds)")
    output_text = gr.Markdown()

    start_button = gr.Button("Start Live Evaluation 🎙️")
    start_button.click(fn=live_analyze, inputs=duration, outputs=output_text)

demo.launch()


Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://eaac780effa0ed50a7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# ======================================================
#  FIXED LIVE RECORDING — USING GRADIO MICROPHONE INPUT
# ======================================================

!pip install gradio==3.50.2 soundfile librosa transformers accelerate ffmpeg-python

import librosa, uuid, os
import gradio as gr
from transformers import pipeline

# ============================
# LOAD WHISPER MODEL
# ============================
ASR_MODEL = "openai/whisper-small"   # later: "prob12/mentorai-whisper-indian"

asr = pipeline(
    "automatic-speech-recognition",
    model=ASR_MODEL,
    chunk_length_s=20
)

# ============================
# SCORING FUNCTION
# ============================
def score_teaching(text, duration):
    words = len(text.split())
    wpm = (words / duration) * 60 if duration > 0 else 1

    clarity = max(1, min(10, 10 - (text.lower().count("um") + text.lower().count("uh"))))
    pace = max(1, min(10, wpm / 13))
    richness = min(10, words / 50)
    confidence = max(1, min(10, 10 - abs(wpm - 130) / 13))

    final = round((clarity*0.4 + pace*0.2 + richness*0.2 + confidence*0.2), 2)

    return {
        "clarity": round(clarity, 2),
        "pace": round(pace, 2),
        "richness": round(richness, 2),
        "confidence": round(confidence, 2),
        "final": final
    }

# ============================
# LIVE ANALYSIS FUNCTION (Gradio mic input)
# ============================
def analyze_live(audio):

    if audio is None:
        return "❌ No audio recorded!"

    # Save the audio file
    file_path = f"live_{uuid.uuid4()}.wav"
    os.rename(audio, file_path)

    # Load audio for duration
    audio_data, sr = librosa.load(file_path, sr=16000)
    duration = len(audio_data) / sr

    # Transcribe
    result = asr(file_path)
    text = result["text"]

    # Score
    scores = score_teaching(text, duration)

    # Return output
    return f"""
# 🎤 MentorAI Live Teaching Evaluation

### Transcript:
{text}

### Scores:
- Clarity: {scores['clarity']}
- Pace: {scores['pace']}
- Richness: {scores['richrichness']}
- Confidence: {scores['confidence']}

### ⭐ Final Score: **{scores['final']} / 10**
"""


# ============================
# GRADIO UI
# ============================
with gr.Blocks(title="Mentor AI - Live Evaluation") as demo:

    gr.Markdown("# 🎙️ MentorAI — Live Teaching Evaluation")
    gr.Markdown("Record audio using the microphone below:")

    audio_input = gr.Audio(source="microphone", type="filepath")
    output_text = gr.Markdown()

    btn = gr.Button("Analyze Recording")
    btn.click(analyze_live, inputs=audio_input, outputs=output_text)

demo.launch()



Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ea3b68013e41e67cbc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
from scipy.io.wavfile import write
import numpy as np

sr = 16000  # Sample rate
duration = 5  # Duration in seconds
f0 = 440.0  # Frequency of sine wave

t = np.linspace(0, duration, int(sr * duration), endpoint=False)
waveform = 0.5 * np.sin(2 * np.pi * f0 * t) # Simple sine wave

sample_audio_path = 'sample_audio.wav'
write(sample_audio_path, sr, waveform.astype(np.float32))

print(f"Generated sample audio file at: {sample_audio_path}")

Generated sample audio file at: sample_audio.wav


In [ ]:
# Call ui_upload with the generated sample audio file
result, pdf_report = ui_upload({'name': sample_audio_path})
print(result)
print(f"Generated PDF report: {pdf_report}")


# 🧑‍🏫 MentorAI Teaching Evaluation Report
### **Final Score: 6.64 / 10**

**Clarity:** 10.0  
**Pace:** 10  
**Confidence:** 1  
**Engagement:** 7.0  

---

### **Transcript**
 Beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, beep, 

In [ ]:
def analyze_live_gradio_input(audio_filepath):
    print(f"DEBUG: analyze_live_gradio_input called with audio_filepath: {audio_filepath}")
    if audio_filepath is None:
        print("DEBUG: No audio recorded, returning error.")
        return "❌ No audio recorded!"

    try:
        # Use the existing analyze_file for comprehensive analysis
        # Since it's live audio, there's no video_path, so extract_engagement will use its default.
        print("DEBUG: Calling analyze_file...")
        text, scores, duration, micro = analyze_file(audio_filepath)
        print("DEBUG: analyze_file completed.")

        # Format the results for Gradio output
        output = f"""
# 🎤 MentorAI Live Teaching Evaluation

### ⏱ Duration: {round(duration, 2)} sec
### 📝 Transcript:
{text}

---

### ⭐ Scores
- **Clarity:** {scores['clarity']}
- **Pace:** {scores['pace']}
- **Richness:** {scores['richness']}
- **Confidence:** {scores['confidence']}
- **Engagement:** {scores['engagement']}

### 🎯 Final Score: **{scores['final_score']} / 10**
"""
        print("DEBUG: Returning formatted output.")
        return output
    except Exception as e:
        import traceback
        tb = traceback.format_exc()
        print(f"DEBUG: Caught exception in analyze_live_gradio_input: {e}")
        print(tb)
        return f"❌ Live Analysis Error:\n{str(e)}\n\nTraceback:\n{tb}"

In [ ]:
# ============================================================
# 🔧 MENTOR AI — FIX PATCH CELL
# This cell fixes all errors in Upload Mode & Live Mode
# ============================================================

# 1️⃣ Install ffmpeg and libsndfile1 (fix video/audio extraction errors)
!apt-get update
!apt-get install -y ffmpeg libsndfile1

# 2️⃣ File handling fix — Gradio passes dict/filepath differently
def safe_file_path(file):
    if file is None:
        return None
    if isinstance(file, dict) and "name" in file:
        return file["name"]
    if hasattr(file, "name"):
        return file.name
    return file

# 3️⃣ Fix for deprecated librosa.write_wav → use soundfile instead
import soundfile as sf
def save_audio_segment(seg, sr, seg_path):
    sf.write(seg_path, seg, sr)

# 4️⃣ Fix Whisper pipeline crashes — wrap in try/except
def whisper_transcribe(audio_path, asr):
    try:
        return asr(audio_path)["text"]
    except Exception as e:
        return f"❌ Whisper Error: {str(e)}"

# 5️⃣ Force Gradio to open outside Colab iframe (mic allowed)
GRADIO_LAUNCH_KWARGS = {"share": True}

print("✅ MentorAI Patch Cell Applied Successfully!")
print("Now re-run your Upload & Live Gradio cells.")

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
def debug_wrapper(func, *args):
    try:
        return func(*args)
    except Exception as e:
        import traceback
        tb = traceback.format_exc()
        return f"❌ REAL ERROR:\n{tb}"


In [ ]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor
import os

model_name = "openai/whisper-small"  # base model you fine-tuned
save_path = "/content/mentorai-whisper-indian"

# Load processor components
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
tokenizer = WhisperTokenizer.from_pretrained(model_name)
processor = WhisperProcessor(feature_extractor, tokenizer)

# Save the processor
feature_extractor.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
processor.save_pretrained(save_path)


# Task
Got it. I'll modify the `segment_audio` function in cell `Fb5zhchk445z` to use `save_audio_segment` to write audio segments instead of the deprecated `librosa.output.write_wav` method.

## Update Audio Segmentation

### Subtask:
Modify the `segment_audio` function in cell `Fb5zhchk445z` to use `save_audio_segment` from the patch cell for writing audio segments. This addresses the deprecated `librosa.output.write_wav`.


## Summary:

### Data Analysis Key Findings
*   The `segment_audio` function was updated to replace the deprecated `librosa.output.write_wav` with `save_audio_segment` for writing audio segments.

### Insights or Next Steps
*   This modification ensures the code uses current and supported libraries, preventing potential issues arising from deprecated functions.


# Task
Define a new function `analyze_live_gradio_input` that accepts a Gradio microphone filepath, performs comprehensive analysis using the existing `analyze_file` function (ensuring `extract_engagement` provides a default if no video is present), and formats the results for display within the Gradio interface, replacing the previous `live_analyze` functions.

## Create Live Analysis Function

### Subtask:
Define a new function `analyze_live_gradio_input` that takes a Gradio microphone filepath, uses `analyze_file` for comprehensive analysis (including `extract_engagement` which will return a default if no video is present), and formats the results for Gradio output.


## Summary:

### Data Analysis Key Findings
*   A new function, `analyze_live_gradio_input`, has been created to process live audio input from a Gradio microphone.
*   This function integrates the comprehensive analysis capabilities of the `analyze_file` function, ensuring consistency with existing analysis workflows.
*   It specifically handles cases where no video is present for engagement analysis, providing a default output as part of the overall analysis.
*   The output of the function is formatted for direct compatibility and display within a Gradio interface.

### Insights or Next Steps
*   This new function streamlines the live analysis workflow by consolidating previous `live_analyze` functions into a single, robust entry point for Gradio microphone inputs.
*   The structured output for Gradio facilitates a user-friendly experience for real-time analysis demonstrations or applications.


# Task
Modify the `analyze_live_gradio_input` function in cell `2783d5dc` to attempt to load the audio file using `librosa.load` and return a message indicating success or a specific error if `librosa.load` fails. Then, re-test the live mode in the Gradio interface and report the output message.

## Isolate Audio Loading Error

### Subtask:
Modify the `analyze_live_gradio_input` function in cell `2783d5dc` to only attempt to load the audio file using `librosa.load`. It should return a success message if loaded, or a specific error message if `librosa.load` fails.


**Reasoning**:
The subtask requires modifying the `analyze_live_gradio_input` function in cell `2783d5dc` to isolate and test the `librosa.load` functionality. This involves updating the function's code to only attempt audio loading and return specific success or failure messages.



In [ ]:
def analyze_live_gradio_input(audio_filepath):
    if audio_filepath is None:
        return "❌ No audio recorded!"

    try:
        audio_data, sr = librosa.load(audio_filepath, sr=16000)
        return f"✅ Audio loaded successfully from: {audio_filepath}"
    except Exception as e:
        return f"❌ Failed to load audio with librosa.load: {str(e)}"

## Final Task

### Subtask:
Instruct the user to re-test the live mode in the Gradio interface and report the output message from the simplified `analyze_live_gradio_input` function.


## Summary:

### Q&A
The solving process successfully modified the `analyze_live_gradio_input` function to isolate the `librosa.load` operation and return appropriate messages. However, the provided solving process does not include the re-testing of the live mode in the Gradio interface and therefore cannot report the output message from that specific test.

### Data Analysis Key Findings
*   The `analyze_live_gradio_input` function was modified to specifically attempt loading an audio file using `librosa.load` with a sampling rate of 16000.
*   Upon successful loading, the function now returns a success message: `✅ Audio loaded successfully from: {audio_filepath}`.
*   If `librosa.load` encounters an error, the function returns a specific error message including the exception details: `❌ Failed to load audio with librosa.load: {str(e)}`.

### Insights or Next Steps
*   The next crucial step is to re-test the modified `analyze_live_gradio_input` function within the Gradio interface's live mode and observe the actual output messages to confirm its behavior with real audio inputs or potential non-audio files.
